In [1]:
# import libraries
# lib for pre trained model 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
# lib for layers
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
# lib for connecting pretrained and created model
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
#lib for compile
from tensorflow.keras.optimizers import Adam
# lib for pre processing
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
# lib for spliting i/p and o/p
from sklearn.model_selection import train_test_split
# lib for metrics
from sklearn.metrics import classification_report
# other lib
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os

In [2]:
# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
# to assign the dataset path
ap.add_argument("-d", "--dataset", type=str, help="path to input dataset")
# to assign for output
ap.add_argument("-p", "--plot", type=str, default="plot.png",help="path to output loss/accuracy plot")
# for model
ap.add_argument("-m", "--model", type=str,default="mask_detectorch.model",help="path to output face mask detector model")
args, unknown = ap.parse_known_args()

In [4]:
# get the list of images in the dataset directory
print("[INFO] loading images...")
imagePaths = list(paths.list_images(r'C:/HOPE/Deep Learning/Face/dataset'))
data =[]
labels = []

# loop over image path
for imagePath in imagePaths:
    # extract the class name from the file name bocz here we have folder data 
    label = imagePath.split(os.path.sep)[-2]
    
    # load the input image(224,224) and preprocess it
    image = load_img(imagePath, target_size=(224,224))
    image = img_to_array(image)
    image = preprocess_input(image)
    
    #append it to the list created --> data and label
    data.append(image)
    labels.append(label)
    # print(data)
    # print(labels)

# convert the data and labels lists 

data = np.array(data,dtype ="float32")
labels = np.array(labels)
# print(data)
# print(labels)

# perform one hot encoding on labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)
# print(labels)

[INFO] loading images...


C:\Anaconda\Lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [5]:
# split the data into training and testing
(trainx, testx, trainy, testy)=train_test_split(data,labels,test_size=0.20,
                                                stratify=labels,random_state=42)

In [6]:
#construct the trianing image generator for data augementation

aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range= 0.2,
    shear_range= 0.15,
    horizontal_flip= True,
    fill_mode= 'nearest'
    )

In [7]:
# load the pre trained  model
basemodel = MobileNetV2(weights ='imagenet',include_top = False,
                        input_tensor = Input(shape = (224,224,3)))

# develope a head model to place in the top of the model
headmodel = basemodel.output
headmodel = AveragePooling2D(pool_size =(7,7))(headmodel)
headmodel = Flatten(name = 'flatten')(headmodel)
headmodel = Dense(128,activation ='relu')(headmodel)
headmodel = Dropout(0.5)(headmodel)
headmodel = Dense(2,activation ='softmax')(headmodel)

# building actual model by combining basemodel and headmodel of FC and that will be the training model
model = Model(inputs =basemodel.input,outputs = headmodel)


C:\Users\kathi\AppData\Local\Temp\ipykernel_10132\2846588376.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  basemodel = MobileNetV2(weights ='imagenet',include_top = False,


In [8]:
# loop all over layers in the basemodel and freeze it to stop the update

# mandatory step in all pre trained model program

for layer in basemodel.layers:
    layer.trainable =False

In [9]:
# compile model

print("[INFO] Compiling model....")
# initialize the initial learning rate, number of epochs to train for,
# and batch size
INIT_LR = 1e-4
EPOCHS = 20
BS = 32
opt = Adam(learning_rate=INIT_LR)
model.compile(loss="binary_crossentropy",optimizer =opt,
              metrics =["accuracy"])

[INFO] Compiling model....


In [10]:
# calculating steps per epoch and validation steps
steps_per_epoch = len(trainx)//BS
validation_steps = len(testx)//BS

# Use .repeat() to ensure the dataset generates enough batches
train_data_gen = aug.flow(trainx, trainy, batch_size=BS)
validation_data_gen = ImageDataGenerator().flow(testx, testy, batch_size=BS)


In [11]:
#train the model

print("[INFO] training ahead....")
H = model.fit(train_data_gen,
              steps_per_epoch=steps_per_epoch,
              validation_data = validation_data_gen,
              validation_steps=validation_steps,
              epochs =EPOCHS)

[INFO] training ahead....
Epoch 1/20


C:\Anaconda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


96/96 ━━━━━━━━━━━━━━━━━━━━ 75s 731ms/step - accuracy: 0.7544 - loss: 0.5828 - val_accuracy: 0.9844 - val_loss: 0.1516
Epoch 2/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9062 - loss: 0.2433 - val_accuracy: 1.0000 - val_loss: 0.0612
Epoch 3/20


C:\Anaconda\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


96/96 ━━━━━━━━━━━━━━━━━━━━ 71s 722ms/step - accuracy: 0.9582 - loss: 0.1669 - val_accuracy: 0.9831 - val_loss: 0.0840
Epoch 4/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 698us/step - accuracy: 0.9688 - loss: 0.1235 - val_accuracy: 1.0000 - val_loss: 0.0101
Epoch 5/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 69s 700ms/step - accuracy: 0.9744 - loss: 0.1012 - val_accuracy: 0.9883 - val_loss: 0.0640
Epoch 6/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 599us/step - accuracy: 0.9688 - loss: 0.1531 - val_accuracy: 1.0000 - val_loss: 0.0490
Epoch 7/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 68s 697ms/step - accuracy: 0.9753 - loss: 0.0831 - val_accuracy: 0.9883 - val_loss: 0.0577
Epoch 8/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 650us/step - accuracy: 1.0000 - loss: 0.0245 - val_accuracy: 1.0000 - val_loss: 0.0037
Epoch 9/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 69s 699ms/step - accuracy: 0.9862 - loss: 0.0551 - val_accuracy: 0.9909 - val_loss: 0.0484
Epoch 10/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 622us/step - accuracy: 0.9688 - loss: 0.0551 - val_accuracy: 1.0000 -

In [12]:
# test prediction
print("['INFO] evaluating model...")
predIdxs =model.predict(testx,batch_size=BS)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability

predIdxs = np.argmax(predIdxs, axis =1)
# print(predIdxs)


['INFO] evaluating model...
25/25 ━━━━━━━━━━━━━━━━━━━━ 14s 506ms/step


In [13]:
# classification report
print(classification_report(testy.argmax(axis =1), predIdxs, target_names=lb.classes_))


              precision    recall  f1-score   support

   with_mask       0.99      0.99      0.99       384
without_mask       0.99      0.99      0.99       386

    accuracy                           0.99       770
   macro avg       0.99      0.99      0.99       770
weighted avg       0.99      0.99      0.99       770



In [14]:

#Save the model
print("[INFO] saving the mask detector model...")
model.save('mask_detector.h5')


[INFO] saving the mask detector model...
